# Preparación del dataset "Wikipedia Movie Plots"

## Nombre: Michael Pillaga


## PARTE 1: RECUPERACION CON TF-IDF

### PASO 1: CARGAR LOS DATOS
Se carga el archivo **`wiki_movie_plots_deduped.csv`** en un **DataFrame de pandas** para su posterior análisis utilizando **TF-IDF** y **similitud de coseno**. Se imprimen las columnas disponibles en el dataset y se muestran algunas filas iniciales para verificar que los datos estén correctamente cargados y listos para procesar.


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el dataset desde un archivo CSV
# Asegúrate de que el archivo esté en la misma carpeta que tu notebook o ajusta la ruta
file_path = "./wiki_movie_plots_deduped.csv"
df = pd.read_csv(file_path)

# Verifica las columnas del dataset
print("Columnas del dataset:", df.columns)

# Opcional: muestra algunas filas para asegurarte de que los datos están cargados correctamente
print(df.head())


Columnas del dataset: Index(['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast',
       'Genre', 'Wiki Page', 'Plot'],
      dtype='object')
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansa

### PASO 2: NORMALIZACION DEL TEXTO

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
# Descargar recursos necesarios de NLTK
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Saitama\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True


Se inicializa un lematizador y un conjunto de **stopwords** en inglés para limpiar y normalizar el texto de las **tramas** de películas. El texto se convierte a minúsculas, se tokeniza, se eliminan las **stopwords** y cada palabra se lematiza antes de reconstruir el texto normalizado. Los resultados se almacenan en la columna **`Normalized_Plot`** y se imprimen para su verificación.


In [5]:
# Inicializar lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    # Convertir a minúsculas y tokenizar
    tokens = text.lower().split()
    # Eliminar stopwords y lematizar
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Reconstruir el texto normalizado
    return " ".join(tokens)

# Aplicar normalización al texto de las tramas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Verificar los primeros resultados normalizados
print(df[['Title', 'Normalized_Plot']].head())


                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                     Normalized_Plot  
0  bartender working saloon, serving drink custom...  
1  moon, painted smiling face hang park night. yo...  
2  film, minute long, composed two shots. first, ...  
3  lasting 61 second consisting two shots, first ...  
4  earliest known adaptation classic fairytale, f...  


### PASO 3: CONFIGURAR TF IDF

Ahora se configura y utiliza un **vectorizador TF-IDF** para convertir los **plots normalizados** de las películas en una **matriz de representaciones vectoriales**. Se eliminan automáticamente las **stopwords** en inglés y se limita el número de características a **5000** términos más relevantes. Finalmente, se imprime la dimensión de la matriz, la cual indica la cantidad de documentos y términos vectorizados.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Configurar el vectorizador TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

# Generar la matriz TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['Normalized_Plot'])

# Verificar las dimensiones de la matriz
print(f"Matriz TF-IDF generada con dimensiones: {tfidf_matrix.shape}")


Matriz TF-IDF generada con dimensiones: (34886, 5000)


### PASO 4: REALIZAR CONSULTAS

Aqui se evalua la relevancia de documentos utilizando **TF-IDF** y **similitud de coseno**. Primero, normaliza la consulta eliminando stopwords y aplicando lematización. Luego, vectoriza la consulta y calcula la similitud entre el **vect


In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Función para normalizar consultas
def normalize_query(query):
    tokens = query.lower().split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Función para evaluar los resultados con TF-IDF
def evaluate_results(query, vectorizer, tfidf_matrix, top_n=5):
    # Normalizar la consulta antes de vectorizarla
    normalized_query = normalize_query(query)
    # Vectorizar la consulta normalizada
    query_vec = vectorizer.transform([normalized_query])
    # Calcular similitud del coseno
    cosine_similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()
    # Obtener los índices de los documentos más similares
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    # Recuperar los documentos con sus puntuaciones
    results = df.iloc[top_indices].copy()
    results['Similarity_Score'] = cosine_similarities[top_indices]
    return results[['Title', 'Plot', 'Similarity_Score']]

# Realizar una consulta
query = "space adventure with dinosaurs"  # Cambia esto por tu consulta
evaluated_results = evaluate_results(query, vectorizer, tfidf_matrix)

# Mostrar los resultados evaluados
print("Documentos recuperados con puntuaciones de similitud:")
print(evaluated_results)



Documentos recuperados con puntuaciones de similitud:
                                Title  \
22892    The Thousand Faces of Dunjia   
12568                    Theodore Rex   
34882              Çalgı Çengi İkimiz   
33715  Space Sheriff Gavan: The Movie   
12103  We're Back! A Dinosaur's Story   

                                                    Plot  Similarity_Score  
22892  The film follows a group of swordsmen's advent...          0.382771  
12568  In an alternate futuristic society where human...          0.373772  
34882  Two musicians, Salih and Gürkan, described the...          0.353538  
33715  Fulfilling their fifteen-year-old childhood dr...          0.337162  
12103  In present-day New York City, an Eastern blueb...          0.329330  


## PARTE 2: RECUPERACION CON BM25

### PASO 1: Verificar conexion con elasticsearch

In [8]:
!pip install elasticsearch


Aqui se establecera una conexión con un clúster de **Elasticsearch** en la URL **`http://localhost:9200`** utilizando la biblioteca de Python **`Elasticsearch`**. Luego, verifica si la conexión fue exitosa mediante el método **`ping()`**, mostrando un mensaje correspondiente para confirmar o detectar errores en la conexión.


In [9]:
from elasticsearch import Elasticsearch

# Conexión al cliente Elasticsearch
es = Elasticsearch("http://localhost:9200")

# Verificar si está conectado
if es.ping():
    print("Conexión exitosa a Elasticsearch")
else:
    print("Error al conectar con Elasticsearch")


Conexión exitosa a Elasticsearch


### Verificar que el indice existe

In [10]:
# Listar los índices existentes con argumentos nombrados
indices = es.indices.get_alias(index="*")
print("Índices existentes:")
for index_name in indices:
    print(index_name)


Índices existentes:
movies


### PASO 2: Realizar consultas

Este código realiza una consulta utilizando **BM25** en **Elasticsearch**. Busca en los campos **`title`** y **`plot`** del índice especificado (por ejemplo, `movies`). Los resultados recuperados incluyen el **título**, un fragmento del **plot** (limitado a 10 palabras), y el **puntaje BM25** asignado por Elasticsearch. La función retorna y muestra los documentos más relevantes basados en el ranking de BM25.


In [11]:
# Función para realizar consultas con BM25
def search_query_bm25(query, index_name, size=5):
    body = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title", "plot"]  # Campos donde buscar
            }
        }
    }
    response = es.search(index=index_name, body=body, size=size)
    
    # Mostrar resultados
    results = []
    for result in response["hits"]["hits"]:
        title = result['_source']['title']
        plot = " ".join(result['_source']['plot'].split()[:10])  # Limitar a 10 palabras del plot
        score = result['_score']
        results.append({"Title": title, "Plot": plot, "BM25_Score": score})
    return results

# Realizar una consulta con BM25
query = "space adventure with dinosaurs"  # Cambia esto por tu consulta
index_name = "movies"  # Nombre del índice en Elasticsearch
bm25_results = search_query_bm25(query, index_name)

# Mostrar los resultados
print("Documentos recuperados con BM25:")
for result in bm25_results:
    print(f"Title: {result['Title']}")
    print(f"Plot: {result['Plot']}...")
    print(f"BM25 Score: {result['BM25_Score']}")
    print("-" * 50)


C:\Users\Saitama\AppData\Local\Temp\ipykernel_3372\2212728940.py:11: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=index_name, body=body, size=size)


Documentos recuperados con BM25:
Title:  Blinky Bill the Movie
Plot: In the town of Greenpatch in Australia, a courageous young...
BM25 Score: 16.69516
--------------------------------------------------
Title: Magic Tree House
Plot: Jack is a shy but confident bookworm and his sister...
BM25 Score: 14.60886
--------------------------------------------------
Title: Unknown Island
Plot: Adventure-seeker Ted Osborne (Phillip Reed) and his fiancee Carole (Virginia...
BM25 Score: 14.467151
--------------------------------------------------
Title: Robot Monster
Plot: Evil Moon robot Ro-Man Extension XJ-2 (Barrows), referred to as...
BM25 Score: 13.367545
--------------------------------------------------
Title: We're Back! A Dinosaur's Story
Plot: In present-day New York City, an Eastern bluebird named Buster...
BM25 Score: 13.172708
--------------------------------------------------


### PASO 3: Evaluar los resultados
 
Este código compara los resultados obtenidos de los métodos de recuperación **TF-IDF** y **BM25**. Para cada resultado, muestra el **título**, el **score de similaridad (TF-IDF)**, el **puntaje BM25**, y un fragmento del **plot**. Esta comparación ayuda a identificar las diferencias en relevancia entre ambos métodos y resalta cómo cada uno prioriza los documentos recuperados.


In [12]:
# Comparar los resultados de BM25 y TF-IDF
def compare_bm25_tfidf(tfidf_results, bm25_results):
    print("\n--- Comparación de Resultados ---")
    for i, (tfidf, bm25) in enumerate(zip(tfidf_results.to_dict('records'), bm25_results)):
        print(f"\nComparación {i + 1}:")
        print(f"TF-IDF - Title: {tfidf['Title']}, Similarity Score: {tfidf['Similarity_Score']}")
        print(f"TF-IDF - Plot: {' '.join(tfidf['Plot'].split()[:10])}...")
        print(f"BM25  - Title: {bm25['Title']}, BM25 Score: {bm25['BM25_Score']}")
        print(f"BM25  - Plot: {bm25['Plot']}...")
        print("-" * 50)

# Llamar la función de comparación
compare_bm25_tfidf(evaluated_results, bm25_results)



--- Comparación de Resultados ---

Comparación 1:
TF-IDF - Title: The Thousand Faces of Dunjia, Similarity Score: 0.3827709009592723
TF-IDF - Plot: The film follows a group of swordsmen's adventures to secretly...
BM25  - Title:  Blinky Bill the Movie, BM25 Score: 16.69516
BM25  - Plot: In the town of Greenpatch in Australia, a courageous young...
--------------------------------------------------

Comparación 2:
TF-IDF - Title: Theodore Rex, Similarity Score: 0.3737716024557386
TF-IDF - Plot: In an alternate futuristic society where humans and anthropomorphic dinosaurs...
BM25  - Title: Magic Tree House, BM25 Score: 14.60886
BM25  - Plot: Jack is a shy but confident bookworm and his sister...
--------------------------------------------------

Comparación 3:
TF-IDF - Title: Çalgı Çengi İkimiz, Similarity Score: 0.3535381884884966
TF-IDF - Plot: Two musicians, Salih and Gürkan, described the adventures of their...
BM25  - Title: Unknown Island, BM25 Score: 14.467151
BM25  - Plot: Adve

## PARTE 3: RECUPERACION CON FAISS

### PASO 1: IMPORTAMOS LIBRERIAS Y CARGAMOS EL DATASET
  
Este código carga el archivo CSV **`wiki_movie_plots_deduped.csv`** en un **DataFrame de pandas** para procesar datos de películas. Una vez cargado, se imprimen las primeras filas del dataset para verificar su contenido y columnas, asegurando que la información como el título y la trama (plot) esté correctamente disponible para su posterior procesamiento.


In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

# Cargar el dataset
file_path = "./wiki_movie_plots_deduped.csv"  # Cambia esto por la ruta del archivo
df = pd.read_csv(file_path)

# Verificar las primeras filas
print("Dataset cargado con las siguientes columnas:")
print(df.head())


Dataset cargado con las siguientes columnas:
   Release Year                             Title Origin/Ethnicity  \
0          1901            Kansas Saloon Smashers         American   
1          1901     Love by the Light of the Moon         American   
2          1901           The Martyred Presidents         American   
3          1901  Terrible Teddy, the Grizzly King         American   
4          1902            Jack and the Beanstalk         American   

                             Director Cast    Genre  \
0                             Unknown  NaN  unknown   
1                             Unknown  NaN  unknown   
2                             Unknown  NaN  unknown   
3                             Unknown  NaN  unknown   
4  George S. Fleming, Edwin S. Porter  NaN  unknown   

                                           Wiki Page  \
0  https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...   
1  https://en.wikipedia.org/wiki/Love_by_the_Ligh...   
2  https://en.wikipedia.org/wiki/Th

### Verificamos dependencias

In [15]:
import torch
import torchvision
from sentence_transformers import SentenceTransformer

print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print("Sentence-Transformers importado correctamente")


PyTorch version: 2.2.2+cpu
Torchvision version: 0.17.2
Sentence-Transformers importado correctamente


### PASO 1: NORMALIZAR TEXTO

Este código aplica un proceso de **normalización de texto** a las tramas (**plots**) de películas contenidas en el DataFrame **`df`**. La normalización incluye:  
- **Convertir el texto a minúsculas.**  
- **Eliminar palabras irrelevantes (stopwords).**  
- **Aplicar lematización** (reducir las palabras a su forma base).  

Los resultados normalizados se almacenan en la nueva columna **`Normalized_Plot`** y se muestran los primeros resultados para su verificación.


In [16]:
# Inicializar lematizador y stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# Función para normalizar el texto
def normalize_text(text):
    tokens = text.lower().split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return " ".join(tokens)

# Aplicar normalización al texto de las tramas
df['Normalized_Plot'] = df['Plot'].apply(normalize_text)

# Verificar los primeros resultados normalizados
print(df[['Title', 'Normalized_Plot']].head())


                              Title  \
0            Kansas Saloon Smashers   
1     Love by the Light of the Moon   
2           The Martyred Presidents   
3  Terrible Teddy, the Grizzly King   
4            Jack and the Beanstalk   

                                     Normalized_Plot  
0  bartender working saloon, serving drink custom...  
1  moon, painted smiling face hang park night. yo...  
2  film, minute long, composed two shots. first, ...  
3  lasting 61 second consisting two shots, first ...  
4  earliest known adaptation classic fairytale, f...  


### PASO 2: GENERAR EMBEDDINGS CON SENTENCETRANSFORMER

Este código carga el modelo preentrenado **`all-MiniLM-L6-v2`** de **`SentenceTransformer`** para generar **embeddings** de las tramas (plots) de películas contenidas en la columna **`Normalized_Plot`** del DataFrame **`df`**. Los embeddings generados son representaciones vectoriales del texto, y el código muestra las dimensiones finales del tensor de embeddings.


In [17]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo preentrenado
model = SentenceTransformer('all-MiniLM-L6-v2')  # Puedes cambiar el modelo si prefieres

# Generar embeddings para las tramas normalizadas
embeddings = model.encode(df['Normalized_Plot'].tolist(), show_progress_bar=True)

# Verificar las dimensiones de los embeddings
print(f"Embeddings generados con dimensiones: {embeddings.shape}")


Batches:   0%|          | 0/1091 [00:00<?, ?it/s]

Embeddings generados con dimensiones: (34886, 384)


### PASO 3: CREAR EL INDICE FAISS

Este código crea un índice **FAISS** utilizando la **distancia Euclidiana (L2)** para realizar búsquedas eficientes de vectores. Primero, obtiene la dimensión de los **embeddings** y luego inicializa el índice. Posteriormente, se agregan todos los **embeddings** al índice y se verifica la cantidad total de vectores almacenados. Esto prepara el índice para futuras consultas rápidas.


In [18]:
# Crear el índice FAISS
import faiss

dimension = embeddings.shape[1]  # Dimensión de los embeddings
index = faiss.IndexFlatL2(dimension)  # L2 = Distancia Euclidiana

# Agregar los embeddings al índice
index.add(embeddings)

# Verificar cuántos vectores hay en el índice
print(f"Cantidad de vectores en el índice: {index.ntotal}")


Cantidad de vectores en el índice: 34886


### PASO 4: REALIZAR CONSULTAS CON FAISS

Este código realiza una consulta en **FAISS**, un índice eficiente para búsquedas de vectores. Primero, genera el **embedding de la consulta** usando el modelo de embeddings proporcionado (`SentenceTransformer`). Luego, busca en el índice FAISS y recupera los **documentos más cercanos** según su distancia. Los resultados incluyen el **título**, las primeras 10 palabras del **plot** y la **distancia** de cada documento respecto a la consulta.


In [19]:
# Función para realizar consultas con FAISS
def search_faiss(query, model, index, top_n=5):
    # Generar embedding de la consulta
    query_embedding = model.encode([query])
    
    # Buscar en el índice FAISS
    distances, indices = index.search(query_embedding, top_n)
    
    # Mostrar resultados
    results = []
    for idx, distance in zip(indices[0], distances[0]):
        title = df.iloc[idx]['Title']
        plot = " ".join(df.iloc[idx]['Plot'].split()[:10])  # Limitar a 10 palabras
        results.append({"Title": title, "Plot": plot, "Distance": distance})
    return results

# Realizar una consulta
query = "space adventure with dinosaurs"  # Cambia esto por tu consulta
faiss_results = search_faiss(query, model, index)

# Mostrar los resultados
print("Documentos recuperados con FAISS:")
for result in faiss_results:
    print(f"Title: {result['Title']}")
    print(f"Plot: {result['Plot']}...")
    print(f"Distance: {result['Distance']}")
    print("-" * 50)


Documentos recuperados con FAISS:
Title: Dinosaurus!
Plot: The movie is about American men building a harbour on...
Distance: 0.9749438166618347
--------------------------------------------------
Title: Dinosaurs! – A Fun-Filled Trip Back in Time!
Plot: The video—with beginning scenes filmed in 1987—begins with a young...
Distance: 0.9765295386314392
--------------------------------------------------
Title: We're Back! A Dinosaur's Story
Plot: In present-day New York City, an Eastern bluebird named Buster...
Distance: 0.9784306287765503
--------------------------------------------------
Title: Yona Yona Penguin
Plot: An animated adventure about three children who travel to a...
Distance: 1.007900595664978
--------------------------------------------------
Title: Jurassic Park III
Plot: Ben Hildebrand and 12-year-old Eric Kirby go parasailing around the...
Distance: 1.0250296592712402
--------------------------------------------------


### PASO 5: COMPARAMOS RESULTADOS CON TF-IDF Y BM25

Este código compara los resultados obtenidos de los métodos de recuperación **TF-IDF**, **BM25** y **FAISS**. Muestra el **título** del documento recuperado junto con su **puntaje de similaridad (TF-IDF)**, **puntaje BM25** y la **distancia (FAISS)** respecto a la consulta. La comparación se realiza iterando sobre los resultados y destacando las diferencias entre ellos.


In [20]:
# Comparar resultados de FAISS con TF-IDF y BM25
def compare_all_results(tfidf_results, bm25_results, faiss_results):
    print("\n--- Comparación de Resultados ---")
    for i, (tfidf, bm25, faiss) in enumerate(zip(tfidf_results.to_dict('records'), bm25_results, faiss_results)):
        print(f"\nComparación {i + 1}:")
        print(f"TF-IDF - Title: {tfidf['Title']}, Similarity Score: {tfidf['Similarity_Score']}")
        print(f"BM25  - Title: {bm25['Title']}, BM25 Score: {bm25['BM25_Score']}")
        print(f"FAISS - Title: {faiss['Title']}, Distance: {faiss['Distance']}")
        print("-" * 50)

# Comparar resultados
compare_all_results(evaluated_results, bm25_results, faiss_results)



--- Comparación de Resultados ---

Comparación 1:
TF-IDF - Title: The Thousand Faces of Dunjia, Similarity Score: 0.3827709009592723
BM25  - Title:  Blinky Bill the Movie, BM25 Score: 16.69516
FAISS - Title: Dinosaurus!, Distance: 0.9749438166618347
--------------------------------------------------

Comparación 2:
TF-IDF - Title: Theodore Rex, Similarity Score: 0.3737716024557386
BM25  - Title: Magic Tree House, BM25 Score: 14.60886
FAISS - Title: Dinosaurs! – A Fun-Filled Trip Back in Time!, Distance: 0.9765295386314392
--------------------------------------------------

Comparación 3:
TF-IDF - Title: Çalgı Çengi İkimiz, Similarity Score: 0.3535381884884966
BM25  - Title: Unknown Island, BM25 Score: 14.467151
FAISS - Title: We're Back! A Dinosaur's Story, Distance: 0.9784306287765503
--------------------------------------------------

Comparación 4:
TF-IDF - Title: Space Sheriff Gavan: The Movie, Similarity Score: 0.3371617834432122
BM25  - Title: Robot Monster, BM25 Score: 13.36754

## PARTE 4: RECUPERACION CON CHROMADB

### PASO 1: CONFIGURAR CHROMADB

In [1]:
import chromadb
from chromadb.utils import embedding_functions

def configurar_chromadb():
    try:
        # Nuevo cliente persistente
        client = chromadb.PersistentClient(path="chromadb_data")  # Ruta de almacenamiento

        # Definir la colección con generación automática de embeddings
        collection = client.get_or_create_collection(
            name="movies",
            embedding_function=embedding_functions.SentenceTransformerEmbeddingFunction("all-MiniLM-L6-v2")
        )

        print("✅ ChromaDB configurado correctamente y la colección 'movies' ha sido creada.")
        return collection

    except Exception as e:
        print(f"❌ Error durante la configuración de ChromaDB: {str(e)}")
        return None

# Ejecuta la configuración
collection = configurar_chromadb()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ ChromaDB configurado correctamente y la colección 'movies' ha sido creada.


### PASO 2: INSERTAR DOCUMENTOS Y EMBEDDINGS

### Cargamos dataset

In [3]:
import pandas as pd

def insertar_documentos(collection, csv_file_path):
    # Cargar el archivo CSV
    data = pd.read_csv(csv_file_path)

    # Verifica que las columnas 'Title' y 'Plot' existan
    if 'Title' not in data.columns or 'Plot' not in data.columns:
        raise ValueError("El archivo CSV no contiene las columnas esperadas.")

    print(f"📂 Procesando {len(data)} documentos...")

    # Insertar todos los documentos (embeddings generados automáticamente)
    for index, row in data.iterrows():
        collection.add(
            documents=[row['Plot']],
            metadatas=[{"title": row['Title']}],
            ids=[str(index)]  # Un ID único por documento
        )
    print(f"✅ {len(data)} documentos añadidos exitosamente con embeddings generados automáticamente.")

# Inserta todos los documentos del archivo CSV
csv_file_path = "./wiki_movie_plots_deduped.csv"
insertar_documentos(collection, csv_file_path)


📂 Procesando 34886 documentos...
✅ 34886 documentos añadidos exitosamente con embeddings generados automáticamente.


## Generación y Actualización de Embeddings Manualmente**

Este código toma un archivo CSV con datos de películas y genera **embeddings** del **plot** utilizando el modelo `SentenceTransformer`. Los embeddings se almacenan en la colección de **ChromaDB** en **lotes de 100 documentos** para optimizar el proceso. Una vez generados, los documentos restantes se actualizan para asegurar que todos tengan sus vectores correspondientes.


In [53]:
import pandas as pd
from sentence_transformers import SentenceTransformer

def generar_y_actualizar_embeddings(collection, csv_file_path):
    # Cargar el modelo de embeddings
    modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

    # Cargar el archivo CSV
    data = pd.read_csv(csv_file_path)

    print(f"📂 Procesando {len(data)} documentos para generar y actualizar embeddings...")

    # Procesar y actualizar en lotes
    batch_size = 100
    batch_ids = []
    batch_embeddings = []

    for index, row in data.iterrows():
        # Generar el embedding del plot
        embedding = modelo_embeddings.encode(row['Plot']).tolist()

        # Almacenar los IDs y embeddings en lotes
        batch_ids.append(str(index))
        batch_embeddings.append(embedding)

        # Si alcanzamos el tamaño del lote, actualizar la colección
        if len(batch_ids) == batch_size:
            collection.update(
                ids=batch_ids,
                embeddings=batch_embeddings
            )
            batch_ids, batch_embeddings = [], []  # Limpiar los lotes

    # Actualizar los documentos restantes
    if batch_ids:
        collection.update(
            ids=batch_ids,
            embeddings=batch_embeddings
        )

    print("✅ Embeddings generados y actualizados manualmente.")

# Ejecutar la función de actualización
csv_file_path = "./wiki_movie_plots_deduped.csv"
generar_y_actualizar_embeddings(collection, csv_file_path)


📂 Procesando 34886 documentos para generar y actualizar embeddings...
✅ Embeddings generados y actualizados manualmente.


## Realización de Consultas con Embeddings

Este código realiza una consulta semántica en **ChromaDB** utilizando un **embedding generado** a partir del texto de la consulta (`"space adventure with dinosaurs"`) mediante el modelo `SentenceTransformer`. Recupera los **10 documentos más cercanos** y muestra su título, plot y la **distancia** calculada entre el embedding de la consulta y los documentos.


In [56]:
from sentence_transformers import SentenceTransformer

def realizar_consulta(collection, consulta, n_resultados=10):
    # Cargar el modelo para generar embeddings
    modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

    # Generar el embedding de la consulta
    query_embedding = modelo_embeddings.encode(consulta).tolist()

    # Buscar los documentos más cercanos e incluir distancias y documentos
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_resultados,
        include=["distances", "documents", "metadatas"]  # Asegurarse de incluir todos los datos necesarios
    )

    # Verificar si hay resultados
    if results is None or not results['documents'][0]:
        print("❌ No se encontraron resultados para la consulta.")
        return None

    # Mostrar resultados
    print("\nResultados de la consulta con distancias:")
    for doc, metadata, distancia in zip(results['documents'][0], results['metadatas'][0], results['distances'][0]):
        print(f"- Título: {metadata['title']}\n  Plot: {doc}\n  Distancia: {distancia:.4f}\n")

# Realiza la consulta con el texto proporcionado
consulta = "space adventure with dinosaurs"
realizar_consulta(collection, consulta, n_resultados=10)



Resultados de la consulta con distancias:
- Título: Dinosaurs! – A Fun-Filled Trip Back in Time!
  Plot: The video—with beginning scenes filmed in 1987—begins with a young boy named Phillip (played by Fred Savage) sitting in his bedroom, listening to loud music, and struggling to find an idea for a class report on a science topic. While struggling to find some ideas—and annoying his mother (offscreen) with his loud music—a song plays on his boom box, titled Mesozoic Mind, and the song provides him with an inspiration for his report: DINOSAURS!
Philip then goes to sleep and has a dream where he discovers that the search for the truth about these magnificent animals and their astonishing 160-million-year success on earth is probably the most fascinating speculation there is. Phillip then finishes his report and presents it to the class. The class report is then covered through the 1980 claymation short Dinosaur by Will Vinton Productions.
  Distancia: 0.9383

- Título: Dinosaurus!
  Plo

In [57]:
import pandas as pd
from sentence_transformers import SentenceTransformer

def realizar_consulta_y_guardar(collection, consulta, n_resultados=10, output_file="resultados_ChromaDB.csv"):
    # Cargar el modelo para generar embeddings
    modelo_embeddings = SentenceTransformer('all-MiniLM-L6-v2')

    # Generar el embedding de la consulta
    query_embedding = modelo_embeddings.encode(consulta).tolist()

    # Buscar los documentos más cercanos e incluir distancias y documentos
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_resultados,
        include=["distances", "documents", "metadatas"]
    )

    # Verificar si hay resultados
    if results is None or not results['documents'][0]:
        print("❌ No se encontraron resultados para la consulta.")
        return None

    # Almacenar resultados en una lista para crear el DataFrame
    resultados_lista = []
    for doc, metadata, distancia in zip(results['documents'][0], results['metadatas'][0], results['distances'][0]):
        resultados_lista.append({
            "Título": metadata['title'],
            "Plot": doc,
            "Distancia": distancia
        })

    # Crear el DataFrame y guardarlo en un archivo CSV
    df = pd.DataFrame(resultados_lista)
    df.to_csv(output_file, index=False)
    print(f"✅ Resultados guardados en {output_file}")

# Realiza la consulta y guarda los resultados
consulta = "space adventure with dinosaurs"
realizar_consulta_y_guardar(collection, consulta, n_resultados=10, output_file="resultados_ChromaDB.csv")


✅ Resultados guardados en resultados_ChromaDB.csv


## Título: Consulta y Almacenamiento de Resultados en un Archivo CSV

Este código realiza una consulta en **ChromaDB** utilizando el **embedding** generado por `SentenceTransformer` a partir de la consulta `"space adventure with dinosaurs"`. Recupera los **10 documentos más cercanos** y guarda los resultados (título, plot y distancia) en un archivo CSV llamado **`resultados_ChromaDB.csv`** para su posterior análisis.


In [22]:
import pandas as pd

# Cargar los 5 primeros resultados de ChromaDB desde el archivo CSV
chromadb_results = pd.read_csv("resultados_ChromaDB.csv").head(5)

# Comparar resultados de FAISS con TF-IDF, BM25 y ChromaDB
def compare_all_results(tfidf_results, bm25_results, faiss_results, chromadb_results):
    print("\n--- Comparación de Resultados ---")
    
    for i, (tfidf, bm25, faiss, chromadb) in enumerate(zip(
        tfidf_results.to_dict('records'), 
        bm25_results, 
        faiss_results, 
        chromadb_results.to_dict('records')
    )):
        # Extraer datos
        tfidf_title = tfidf['Title']
        bm25_title = bm25['Title']
        faiss_title = faiss['Title']
        chromadb_title = chromadb['Título']

        tfidf_score = tfidf['Similarity_Score']
        bm25_score = bm25['BM25_Score']
        faiss_distance = faiss['Distance']
        chromadb_distance = chromadb['Distancia']

        # Imprimir resultados de la comparación
        print(f"\nComparación {i + 1}:")
        print(f"TF-IDF  - Title: {tfidf_title}, Similarity Score: {tfidf_score}")
        print(f"BM25    - Title: {bm25_title}, BM25 Score: {bm25_score}")
        print(f"FAISS   - Title: {faiss_title}, Distance: {faiss_distance:.4f}")
        print(f"ChromaDB - Title: {chromadb_title}, Distance: {chromadb_distance:.4f}")

        # Comparar títulos
        if tfidf_title == chromadb_title and faiss_title == chromadb_title and bm25_title == chromadb_title:
            print("✅ Coincidencia completa de títulos entre todos los métodos.")
        else:
            print("⚠️ Diferencias en los títulos detectadas:")
            if tfidf_title != chromadb_title:
                print(f"  - Diferencia entre TF-IDF y ChromaDB: {tfidf_title} vs {chromadb_title}")
            if bm25_title != chromadb_title:
                print(f"  - Diferencia entre BM25 y ChromaDB: {bm25_title} vs {chromadb_title}")
            if faiss_title != chromadb_title:
                print(f"  - Diferencia entre FAISS y ChromaDB: {faiss_title} vs {chromadb_title}")

        print("-" * 50)

# Llamar a la función de comparación
compare_all_results(evaluated_results, bm25_results, faiss_results, chromadb_results)



--- Comparación de Resultados ---

Comparación 1:
TF-IDF  - Title: The Thousand Faces of Dunjia, Similarity Score: 0.3827709009592723
BM25    - Title:  Blinky Bill the Movie, BM25 Score: 16.69516
FAISS   - Title: Dinosaurus!, Distance: 0.9749
ChromaDB - Title: Dinosaurs! – A Fun-Filled Trip Back in Time!, Distance: 0.9383
⚠️ Diferencias en los títulos detectadas:
  - Diferencia entre TF-IDF y ChromaDB: The Thousand Faces of Dunjia vs Dinosaurs! – A Fun-Filled Trip Back in Time!
  - Diferencia entre BM25 y ChromaDB:  Blinky Bill the Movie vs Dinosaurs! – A Fun-Filled Trip Back in Time!
  - Diferencia entre FAISS y ChromaDB: Dinosaurus! vs Dinosaurs! – A Fun-Filled Trip Back in Time!
--------------------------------------------------

Comparación 2:
TF-IDF  - Title: Theodore Rex, Similarity Score: 0.3737716024557386
BM25    - Title: Magic Tree House, BM25 Score: 14.60886
FAISS   - Title: Dinosaurs! – A Fun-Filled Trip Back in Time!, Distance: 0.9765
ChromaDB - Title: Dinosaurus!, Distan

## Parte 5: Comparación de Resultados

### Relevancia - Ventajas y Limitaciones

## 🔍 Comparación General entre los Métodos

### **1. TF-IDF**
- Recupera documentos con títulos muy diferentes a los de **ChromaDB** y **FAISS**.
- Sus **scores de similaridad** son moderadamente bajos (por debajo de `0.4`), lo que indica que podría no estar encontrando documentos muy relevantes.
- Parece no manejar bien las **relaciones semánticas profundas** y funciona mejor cuando hay **coincidencias literales de términos**.

---

### **2. BM25**
- También muestra muchas diferencias en los títulos recuperados.
- Sus **puntajes BM25** son relativamente altos (`~13-16`), pero los títulos de los resultados difieren mucho de los de **FAISS** y **ChromaDB**.
- Al igual que **TF-IDF**, parece no capturar bien las relaciones semánticas profundas, sino que se basa más en la **frecuencia de los términos**.

---

### **3. FAISS (con embeddings)**
- Aunque los títulos recuperados por **FAISS** suelen estar más relacionados semánticamente con la consulta, muestra diferencias respecto a **ChromaDB**.
- Las **distancias** están en el rango de `0.97-1.02`, lo que sugiere que los documentos encontrados son **moderadamente similares** a la consulta.
- Tiene un rendimiento mejor que **TF-IDF** y **BM25** en cuanto a **capturar relaciones semánticas**.

---

### **4. ChromaDB (con embeddings)**
- Los documentos encontrados por **ChromaDB** muestran una **mayor consistencia** en las distancias y parecen estar relacionados semánticamente.
- Las distancias están **ligeramente por debajo** de las de **FAISS**, lo que puede indicar que está recuperando documentos un poco más relevantes.
- A menudo, **ChromaDB** parece encontrar títulos más **directamente relacionados con la consulta** en comparación con **TF-IDF** y **BM25**.

---

##  **Resumen de la Comparación**

| Método    | Relevancia semántica | Coincidencia literal | Observación clave                                                      |
|-----------|---------------------|----------------------|------------------------------------------------------------------------|
| **TF-IDF**  | Baja                | Alta                 | Funciona bien cuando hay coincidencias literales, pero falla en consultas semánticas profundas. |
| **BM25**   | Baja                | Media                | Se basa en la frecuencia de términos, lo que limita su relevancia en consultas semánticas.      |
| **FAISS**  | Alta                | Media                | Captura bien relaciones semánticas, pero puede diferir en distancias respecto a ChromaDB.       |
| **ChromaDB** | Muy alta            | Media                | Tiene el mejor rendimiento en términos de relevancia semántica y consistencia en resultados.     |

---